# Create 2D horizontal plots
**Author: Jun Sasaki  Coded on 2025-01-13  Updated on 2025-04-27**<br>
Create a 2D horizontal contour plots. Customization can be made by defining `post_process_func`, or `post_process_func=None` (default) without customization. 

```Python
def post_process_func(ax, da, time):
    """
    Example of post_process_func for customizing plot (e.g., add text or markers)
    
    Parameters:
    - ax: matplotlib axis.
    - da: DataArray (optional and used for dynamic customizing).
    - time: Frame time (optional and used for dynamic customizing).
    """
```

In [ ]:
import os
import numpy as np
import pandas as pd
from xfvcom import FvcomDataLoader, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import FrameGenerator
from xfvcom.helpers_utils import apply_xlim_ylim
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.tri as tri

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

png_dir = "PNG"
os.makedirs(png_dir, exist_ok=True)

## Prepare FvcomDataLoader instance of `fvcom` using FVCOM output netcdf.
Dataset is `fvcom.ds`.

In [ ]:
# Loading FVCOM output netcdf
base_path = "~/Github/TB-FVCOM/goto2023"
#base_path =  "/home/pj24001722/ku40003295/Ersem_TokyoBay/output_5times"
base_path = os.path.expanduser(base_path)
# List of netcdf files convenient to switch to another netcdf by specifying its index
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
#ncfiles = ["tb_0001.nc"]
index_ncfile = 0
ncfile_path = f"{base_path}/output/{ncfiles[index_ncfile]}"
# Loading FVCOM input open boundary node file of tst_obc.dat.
obcfile_path =  f"{base_path}/input/TokyoBay18_obc.dat"
# Create an instance of FvcomDataLoader where fvcom.ds is a Dataset
fvcom = FvcomDataLoader(ncfile_path=ncfile_path, obcfile_path=obcfile_path, time_tolerance=5)
cfg = FvcomPlotConfig(figsize=(6, 8))
plotter = FvcomPlotter(fvcom.ds, cfg)

In [ ]:
print(fvcom.ds.lon.values)
fvcom.ds

## Bathymetry and mesh plot
Plots can be modified by applying `post_process_func` (optional). 

In [ ]:
def custom_plot_bathymetry(ax):
    """
    Customizing the plot by updating the ax

    Parameters:
    - ax: matplotlib axis
    """

    # Further customization can be added.
    ax.set_title("FVCOM bathymetry and mesh")
    transform=ccrs.PlateCarree()
    lon = fvcom.ds.lon.values
    lat = fvcom.ds.lat.values

    # Set xlim and ylim manually.
    xlim = ("139:40:00", "140:00:00")
    ylim = ("35:12:00", "35:50")
    xlim = (139.9, 140.0)
    ylim = (35.6, 35.7)
    #xlim = (139.7, 140.1)
    #ylim = (35, 35.6)

    apply_xlim_ylim(ax, xlim, ylim, is_cartesian=False)
    # Get the current map region to clip texts outside the map
    xmin, xmax, ymin, ymax = ax.get_extent(crs=ccrs.PlateCarree())
    for n in range(len(fvcom.ds.lon)):
        # No plot outide the map
        if lon[n] < xmin or lon[n] > xmax or lat[n] < ymin or lat[n] > ymax:
            continue
        ax.text(lon[n], lat[n], str(n+1), fontsize=10, color='yellow', ha='center', va='bottom',
                transform=transform, clip_on=True)
        ax.plot(lon[n], lat[n], 'ro', transform=transform, clip_on=True)  # Marker plot at (lon, lat)

# Specify bathymetry "h"
var_name = "h"
da = plotter.ds[var_name]
da = None # only mesh plot

# Set plot_kwargs for `ax.tricontourf()` kwargs, including `**kwargs`.
vmin = 0; vmax = 500  # = None for automated
levels = [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30, 35, 40, 50, 100, 200, 300, 400, 500]
# levels = 20
cmap = "jet"  # "jet", "Blues"
plot_kwargs = {"verbose": False, "vmin": vmin, "vmax": vmax, "levels": levels,
             "cmap": cmap, "with_mesh": True, "coastlines": True, "obclines": True,
             "plot_grid": False, "add_tiles": True, "color": "yellow", "coastline_color": "yellow"}
# plot_kwargs = {} # Default plot
save_path = os.path.join(png_dir, f"{var_name}.png")

ax = FrameGenerator.plot_data(da=da, plotter=plotter, post_process_func=custom_plot_bathymetry, plot_kwargs=plot_kwargs)
fig=ax.figure
ax.set_extent([139.9, 140.0, 35.6, 35.7], crs=ccrs.PlateCarree())
fig.savefig(save_path, bbox_inches='tight', dpi=300)

## Create 2D plot with `post_process_func`.
- 2-D horizontal plot with customization by updating `ax` `if post_process_func is not None`.

In [ ]:
def static_custom_plot(ax):
    """
    Customizing plot by updating ax

    Parameters:
    - ax: matplotlib axis
    """

    # Further customization can be added.
    ax.set_title("Title with Custom Plot")

def dynamic_custom_plot(ax, da, time):
    """
    Plot the corresponding datetime at each frame

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.    
    """
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

# Specify var_name and siglay if any
var_name = "salinity"
time = 20
siglay = 0
da = plotter.ds[var_name][:,siglay,:] # time must be included in da as reusing a tool for animation.
time_str = fvcom.ds.time.isel(time=time).values
time_str = pd.to_datetime(time_str).strftime("%Y%m%d")
# Set plot_kwargs for `ax.tricontourf(**kwargs)`.
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet"}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "Blues", "with_mesh": False, "coastlines": True, "obclines": True,
             "plot_grid": True, "add_tiles": True}
#plot_kwargs={}
save_path = os.path.join(png_dir, f"{var_name}_{time_str}.png")

ax = FrameGenerator.plot_data(da=da, time=time, plotter=plotter,
    save_path=None, post_process_func=dynamic_custom_plot, plot_kwargs=plot_kwargs)
fig=ax.figure
fig.savefig("test10.png")

### Specify xlim and ylim
Suppose `fvcom` and `plotter` instances, `var_name`, `siglay`, `time`, and `save_path` are prepared above.

In [ ]:
def custom_plot(ax, da, time):
    """
    Plot the corresponding datetime at each frame and set xlim and ylim

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.
    """

    # Put datetime text.
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

    # Set xlim and ylim manually.
    xlim = ("139:40:00", "140:00:00")
    ylim = ("35:12:00", "35:50")
    #xlim = (139.7, 140.1)
    #ylim = (35, 35.6)

    apply_xlim_ylim(ax, xlim, ylim, is_cartesian=False)

# Set plot_kwargs for `ax.tricontourf(**kwargs)`.
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True, "coastlines": True}

FrameGenerator.plot_data(da=da, time=time, plotter=plotter,
    save_path=save_path, post_process_func=custom_plot, plot_kwargs=plot_kwargs)